# Validation of the tsib.timeseriesmanager.get_era5 module
Author: Leander Kotzur

Date: 21.09.2019

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import tsib.timeseriesmanager as tsm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import copy
import os

In [3]:
from netCDF4 import Dataset,num2date

In [4]:
import cdsapi


Get weather data

In [5]:
try_data,location = tsm.readTRY()

In [6]:
try_data.head()

,RG,IS,MM,DD,HH,N,WR,WS,T,p,...,RF,W,B,DHI,IK,A,E,IL,GHI,DNI
2010-01-01 00:30:00+01:00,4,1,1,1,1,7,230,5.7,-2.6,1005.3,...,93,70,0,0,1,251,-285,9,0,0.0
2010-01-01 01:30:00+01:00,4,1,1,1,2,7,240,5.7,-3.9,1011.4,...,89,70,0,0,1,275,-291,9,0,0.0
2010-01-01 02:30:00+01:00,4,1,1,1,3,8,260,5.7,-4.6,1016.0,...,90,22,0,0,1,281,-296,9,0,0.0
2010-01-01 03:30:00+01:00,4,1,1,1,4,8,270,5.7,-3.9,1016.2,...,95,22,0,0,1,281,-297,9,0,0.0
2010-01-01 04:30:00+01:00,4,1,1,1,5,8,280,5.5,-3.3,1016.5,...,97,10,0,0,1,283,-299,9,0,0.0


In [7]:
location

{'name': b'Potsdam',
 'latitude': 52.38333333333333,
 'longitude': 13.066666666666666}

## Get single profiles

In [10]:
filename = 'download2.nc'

In [12]:

c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-land',
    {
        'format':'netcdf',
        'variable':[
            '2m_temperature',
            'surface_net_solar_radiation',
            'total_sky_direct_solar_radiation_at_surface'
        ],
        'area':str(location['latitude']) + 
                '/' + str(location['longitude']) +
                '/' + str(location['latitude']+0.1) + 
                '/' + str(location['longitude']+0.1),
        'year':'2014',
        'month':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12'
        ],
        'day':[
            '01','02','03',
            '04','05','06',
            '07','08','09',
            '10','11','12',
            '13','14','15',
            '16','17','18',
            '19','20','21',
            '22','23','24',
            '25','26','27',
            '28','29','30',
            '31'
        ],
        'time':[
            '00:00','01:00','02:00',
            '03:00','04:00','05:00',
            '06:00','07:00','08:00',
            '09:00','10:00','11:00',
            '12:00','13:00','14:00',
            '15:00','16:00','17:00',
            '18:00','19:00','20:00',
            '21:00','22:00','23:00'
        ]
    },
    filename,
)

2019-09-30 21:23:41,431 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-land
2019-09-30 21:23:41,723 INFO Request is queued
2019-09-30 21:23:42,768 INFO Request is running
2019-09-30 22:21:34,304 INFO Request is completed
2019-09-30 22:21:34,306 INFO Downloading http://136.156.133.37/cache-compute-0011/cache/data7/adaptor.mars.internal-1569871422.1639633-29917-15-ea545d2e-ba59-4b89-8638-716d7c1b95f1.nc to download2.nc (69.8K)
2019-09-30 22:21:34,527 INFO Download rate 318.3K/s


Result(content_length=71460,content_type=application/x-netcdf,location=http://136.156.133.37/cache-compute-0011/cache/data7/adaptor.mars.internal-1569871422.1639633-29917-15-ea545d2e-ba59-4b89-8638-716d7c1b95f1.nc)

In [13]:
nc_pvFile = Dataset(filename,'r')
lats = nc_pvFile.variables['latitude'][:]  # extract/copy the data
lons = nc_pvFile.variables['longitude'][:]
lats = lats[:].squeeze()
lons = lons[:].squeeze()

In [ ]:
# initialize a pandas.DataFrame to store all the data
time_index = pd.date_range(
    start=str(year) + "-01-01 00:30:00",
    end=str(year) + "-12-31 23:30:00",
    freq="H",
    tz="Europe/Berlin",
)

In [ ]:
def get_era5(lon, lat, year):
    """
    Get data from the cosmo data set.

    Parameters
    ----------
    lon: float
        Longitude in degree.
    lat: float
        Latitude in degree.
    year: int
        Year from 2010 to 2015 as integer.

    Returns
    -------
    df: pd.DataFrame with the data.
    datapath: Identifier for the weather data.
    """
    try:
        import res
        import netCDF4 as nc
    except ImportError:
        raise ImportError('Internal packages are required.')

    # required variables and a translation to the terms used in tsib
    VARIABLES2TRY = {
        "SWDIFDS_RAD": "DHI",
        "SWDIRS_RAD": "DirectHI",
        "2t": "T",
        "windspeed_10": "WS",
        "sp": "p",
    }

    # get the data allocation
    data = res.weather.CosmoSource(
        [os.path.join(datapath, str(year), "*.nc")], bounds=(lon, lat), verbose=False
    )

    # get the index for the netCDF4 file
    ix = res.weather.CosmoSource.loc2Index(None, (lon, lat))

    # get a filepath for the resulting weather data set, ix[1]
    identifier = "COSMO_Year_" + str(year) + "_ix_" + str(ix[0]) + "_" + str(ix[1])
    filepath = os.path.join(DATA_PATH, "weatherdata", "COSMO", identifier)

    # get file if existing
    if os.path.isfile(filepath + ".csv"):
        df = pd.read_csv(filepath + ".csv", index_col=0, parse_dates=True)
        df.index = df.index.tz_localize("UTC").tz_convert("Europe/Berlin")

    # get data
    else:
        # initialize a pandas.DataFrame to store all the data
        time_index = pd.date_range(
            start=str(year) + "-01-01 00:30:00",
            end=str(year) + "-12-31 23:30:00",
            freq="H",
            tz="Europe/Berlin",
        )
        df = pd.DataFrame(index=time_index)

        # loop over required variables and load the data into the dataframe
        for var in VARIABLES2TRY.keys():
            # load dataset via netCDF4
            ds = nc.Dataset(data.variables.loc[var, "path"])
            df[VARIABLES2TRY[var]] = ds.variables[var][:, ix[0], ix[1]]

        # convert temperature from K to °C
        df["T"] = df["T"] - 273.15

        # get GHI
        df["GHI"] = df["DirectHI"] + df["DHI"]

        # calclulate DNI
        df["DNI"] = calculateDNI(df["DirectHI"], lon, lat)

        # remove leap day
        sel = np.logical_and((time_index.day == 29), (time_index.month == 2))
        df = df.loc[~sel]

        # save as .csv
        df.to_csv(filepath + ".csv")
    return df, identifier

In [14]:
nc_pvFile.variables

{'longitude': <class 'netCDF4._netCDF4.Variable'>
 float32 longitude(longitude)
     units: degrees_east
     long_name: longitude
 unlimited dimensions: 
 current shape = (1,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'latitude': <class 'netCDF4._netCDF4.Variable'>
 float32 latitude(latitude)
     units: degrees_north
     long_name: latitude
 unlimited dimensions: 
 current shape = (1,)
 filling on, default _FillValue of 9.969209968386869e+36 used,
 'time': <class 'netCDF4._netCDF4.Variable'>
 int32 time(time)
     units: hours since 1900-01-01 00:00:00.0
     long_name: time
     calendar: gregorian
 unlimited dimensions: 
 current shape = (8760,)
 filling on, default _FillValue of -2147483647 used,
 't2m': <class 'netCDF4._netCDF4.Variable'>
 int16 t2m(time, latitude, longitude)
     scale_factor: 0.0006891171182362322
     add_offset: 282.8943087617534
     _FillValue: -32767
     missing_value: -32767
     units: K
     long_name: 2 metre temperature
 unlimi

In [ ]:
t2m_raw = nc_pvFile.variables['t2m'][:]

In [ ]:
t2m = t2m_raw[:,0,0]


In [ ]:
len(t2m)

from netCDF4 import Dataset,num2date
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point


preciPlot = nc_pvFile.variables['tp'][:]

pp = preciPlot[0,:,:]


ax1 = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))
clevs = np.arange(min(pp.flatten()),max(pp.flatten())*1000,1)


shear_fill = ax1.contourf(lons,lats,pp*1000,clevs,
                      transform=ccrs.PlateCarree(),cmap=plt.get_cmap('hsv'),
                                                                      linewidth=(10,),levels=100,extend='both')

ax1.coastlines()
ax1.gridlines()
ax1.set_xticks([0, 10,20,30,40,50,60,70,80,90,100], crs=ccrs.PlateCarree())
ax1.set_yticks([0, 10,20,30,40,50,60], crs=ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True,
                               number_format='.0f')
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)
cbar = plt.colorbar(shear_fill, orientation='horizontal')
plt.title('Total precipitation', fontsize=16)
plt.savefig('precip_era.png')